In [1]:
from os import listdir
from os.path import isfile, join

def list_files(my_path):
    onlyfiles = [f for f in listdir(my_path) if isfile(join(my_path, f))]
    
    return onlyfiles

In [2]:
import numpy as np

def del_whitespace(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = 255*(gray < 128).astype(np.uint8) # To invert the text to white
    coords = cv2.findNonZero(gray) # Find all non-zero points (text)
    x, y, w, h = cv2.boundingRect(coords) # Find minimum spanning bounding box
    rect = img[y:y+h, x:x+w] # Crop the image - note we do this on the original image
    return rect

def my_resize(img, ww, hh):
#     img = del_whitespace(img)
    ht, wd, cc= img.shape
    
    wp = ww / wd
    hp = hh / ht
    
    p = wp
    if hp < wp:
        p = hp
    ht = int(ht * p)
    wd = int(wd * p)
    img = cv2.resize(img, (wd, ht))
    
    color = (255,255,255)
    result = np.full((hh,ww,cc), color, dtype=np.uint8)

    # compute center offset
    xx = (ww - wd) // 2
    yy = (hh - ht) // 2

    # copy img image into center of result image
    result[yy:yy+ht, xx:xx+wd] = img
    return result

In [3]:
import cv2
import shutil

def read_transparent_png(filename):
    image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if not hasattr(image, 'shape'):
        return cv2.imread(filename)
    
    if len(image.shape) == 2:
        return np.stack((image,)*3, axis=-1)
    
    if hasattr(image, 'shape') and image.shape[2] == 3:
        return image

    alpha_channel = image[:,:,3]
    rgb_channels = image[:,:,:3]

    # White Background Image
    white_background_image = np.ones_like(rgb_channels, dtype=np.uint8) * 255

    # Alpha factor
    alpha_factor = alpha_channel[:,:,np.newaxis].astype(np.float32) / 255.0
    alpha_factor = np.concatenate((alpha_factor,alpha_factor,alpha_factor), axis=2)

    # Transparent Image Rendered on White Background
    base = rgb_channels.astype(np.float32) * alpha_factor
    white = white_background_image.astype(np.float32) * (1 - alpha_factor)
    final_image = base + white
    return final_image.astype(np.uint8)

def do_resize(source_path, target_path, ww, hh):
    images = list_files(source_path)
    for image in images:
        image_source_path = "{}/{}".format(source_path, image)
        image_target_path = "{}/{}".format(target_path, image)

        img = read_transparent_png(image_source_path)
        if (img is None):            
            shutil.copy(image_source_path, image_target_path)
        else:
            img = my_resize(img, ww, hh)

            cv2.imwrite(image_target_path, img)

In [4]:
source = "./source"
target = "./target"
ww = 600
hh = 200
do_resize(source, target, ww, hh)